In [1]:
import flywheel
import datetime

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/xcpengine-fw')
project = fw.lookup('davis/presurgicalEpilepsy')
design_file = project.get_file('cbf_WT.dsn')

# set date for label
x = datetime.datetime.now()
date_str = '%s-%s-%s %s:%s' % (x.month, x.day, x.year, x.hour, x.minute)

In [34]:
# create subject list
fin = 'patients.txt'
sid_list = [line.rstrip('\n').split()[0] for line in open(fin)]
sid_list

['P01',
 'P02',
 'P03',
 'P04',
 'P05',
 'P06',
 'P07',
 'P08',
 'P09',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37',
 'P38',
 'P39',
 'P40',
 'P41',
 'P42',
 'P43',
 'P44',
 'P45',
 'P46',
 'P47',
 'P48',
 'P49',
 'P50',
 'P51',
 'P52',
 'P53',
 'P54',
 'P55',
 'P56',
 'P57',
 'P58',
 'P59',
 'P60',
 'P61',
 'P62',
 'P63',
 'P64',
 'P65',
 'P66',
 'P67',
 'P68',
 'P69',
 'P70',
 'P71',
 'P72',
 'P73',
 'P74',
 'P75',
 'P76',
 'P77',
 'P78',
 'P79',
 'P80']

In [37]:
for sub in sid_list:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    print('Running analysis for subject %s' % (fw.get(ses.parents['subject']).label))
    
    # get ASL and M0 files
    asl_img = None
    m0_img = None
    for acq in ses.acquisitions():
        if 'ASL' in acq.label:
            asl_container=fw.get(acq.id)
            for f in asl_container.files:
                if '.nii.gz' in f.name:
                    asl_img=f
        if 'M0' in acq.label:
            m0_container=fw.get(acq.id)
            for f in m0_container.files:
                if '.nii.gz' in f.name:
                    m0_img=f
        else:
            pass
        
    for a in ses.analyses:
        if a.files and 'SB_fmriprep' in a.label:
            print(a.label)
            fprep_output = [f for f in a.files if 'fmriprep' in f.name]
            print(fprep_output[0].name)
    
    
    if asl_img is not None and m0_img is not None:
        # set inputs and config for gear
        inputs = {'fmriprepdir': fprep_output[0], 'designfile': design_file, 'img': asl_img, 'm0': m0_img}
        config = {'analysis_type': 'cbf'}
        print("THERE ARE ASL IMAGES FOR " + sub)
        #analysis_id = gear.run(analysis_label='WT_xcp cbf ' + date_str, config=config, inputs=inputs, destination=ses)

Running analysis for subject P01
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P01_5d530dabc3758a006ede81b0.zip
Running analysis for subject P02
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P02_5d530daac3758a006ede81ae.zip
Running analysis for subject P03
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P03_5d530daac3758a006ede81ac.zip
Running analysis for subject P04
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P04_5d530daac3758a0049e0f0b5.zip
Running analysis for subject P05
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P05_5d530daac3758a006ede81aa.zip
Running analysis for subject P06
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P06_5d530daac3758a0045da6077.zip
Running analysis for subject P07
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P07_5d530daac3758a0047ddd5ab.zip
Running analysis for subject P08
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P08_5d530daac3758a0043d84fe5.zip
Running analysis for subject P09
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P09_5d530daac3758a0027d44826.zip
Running analysis for subject P10
SB_f

SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P68_5d530da4c3758a0047ddd593.zip
THERE ARE ASL IMAGES FOR P68
Running analysis for subject P69
SB_fmriprep 08/20/2019 15:22:15
fmriprep_sub-P69_5d5c488bc3758a006eec1a63.zip
Running analysis for subject P70
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P70_5d530dacc3758a0033d47980.zip
THERE ARE ASL IMAGES FOR P70
Running analysis for subject P71
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P71_5d530dacc3758a006ede81b9.zip
THERE ARE ASL IMAGES FOR P71
Running analysis for subject P72
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P72_5d530dacc3758a0027d44828.zip
THERE ARE ASL IMAGES FOR P72
Running analysis for subject P73
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P73_5d530dabc3758a006ede81b4.zip
THERE ARE ASL IMAGES FOR P73
Running analysis for subject P74
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P74_5d530dabc3758a003bd57a88.zip
THERE ARE ASL IMAGES FOR P74
Running analysis for subject P75
SB_fmriprep 8-13-2019 15:21
fmriprep_sub-P75_5d530dabc3758a006ede81b6.zip